# HCN1 Mutation Trans Effect on Transcriptomics

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.utils as al

brain = cptac.Gbm()

### Select Gene

In [2]:
gene = "HCN1"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

### Track all significant comparisons in Dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Transcriptomics

### Generate interacting protein list

Make a call to cptac.utils to get the interacting proteins method, which interacts with the uniprot and string databases to generate a list of known interacting partners with the given gene.

In [6]:
# Use get interacting proteins method to generate list of interacting proteins
interacting_proteins = al.get_interacting_proteins(gene)

# Show interacting protein list
print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)

Interacting Proteins:
MARCH7
HCN2
HCN4
KCNE2
KCTD3
SHKBP1
SCN1A
HCN1
MAP6
KCNA4
SCN8A
KCNQ2
ASTN1
FLNA
OR4A5
KCNA1
GNAT3
NEDD4L
UBAP2L
PEX5L
AP3B2
APBA2
PRSS57
RNF123


### Test for significant comparisons in any of interacting proteins

In [7]:
# Create dataframe in order to do comparisons with wrap_ttest
protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = brain.reduce_multiindex(protdf, flatten=True)
protdf = protdf[protdf.Sample_Status == 'Tumor'] # drop Normal samples

# Create the binary valued column needed to do the comparison
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

# Format the dataframe correctly
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


# Make list of columns to be compared using t-tests
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

# Call wrap_ttest, pass in formatted dataframe
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

# Print results, if anything significant was found
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", True, all_significant_comparisons)
        

Doing t-test comparisons

                                 Comparison   P_Value
0  PEX5L_transcriptomics_ENSG00000114757.17  0.000003
1    HCN2_transcriptomics_ENSG00000099822.2  0.000270





# All Proteins: Transcriptomics 

In [8]:
try:
    print("\nGene: ", gene)

    # Use all proteins

    # Create dataframe in order to do comparisons with wrap_ttest
    protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = brain.reduce_multiindex(protdf, flatten=True) # flatten for specific column names
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    # Create binary column 
    protdf['Label'] = np.where(
                protdf[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

    # Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    # Make list of columns to be compared using t-tests
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    # Call wrap_ttest, pass in formatted dataframe
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    # Print results, if anything significant was found
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  HCN1


Doing t-test comparisons

                                          Comparison       P_Value
0           TSPAN8_transcriptomics_ENSG00000127324.7  3.750814e-12
1           ENPP2_transcriptomics_ENSG00000136960.11  2.178688e-11
2          ELOVL1_transcriptomics_ENSG00000066322.11  5.735624e-11
3          CYCSP23_transcriptomics_ENSG00000253495.1  2.883651e-10
4    RP11-338K13.1_transcriptomics_ENSG00000273267.1  1.191443e-09
..                                               ...           ...
123        NKAIN2_transcriptomics_ENSG00000188580.12  6.467237e-07
124  RP11-350G24.2_transcriptomics_ENSG00000258033.1  7.093768e-07
125  RP11-202K23.1_transcriptomics_ENSG00000233359.1  7.327203e-07
126       GOLGA8J_transcriptomics_ENSG00000179938.12  7.729804e-07
127  RP11-946L20.1_transcriptomics_ENSG00000254241.1  8.202321e-07

[128 rows x 2 columns]





# Print all significant comparisons

In [9]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

,Cancer_Type,Gene,Comparison,Interacting_Protein,P_Value
0,Gbm,HCN1,PEX5L_transcriptomics_ENSG00000114757.17,True,2.743747e-06
1,Gbm,HCN1,HCN2_transcriptomics_ENSG00000099822.2,True,2.701972e-04
0,Gbm,HCN1,TSPAN8_transcriptomics_ENSG00000127324.7,False,3.750814e-12
1,Gbm,HCN1,ENPP2_transcriptomics_ENSG00000136960.11,False,2.178688e-11
2,Gbm,HCN1,ELOVL1_transcriptomics_ENSG00000066322.11,False,5.735624e-11
...,...,...,...,...,...
123,Gbm,HCN1,NKAIN2_transcriptomics_ENSG00000188580.12,False,6.467237e-07
124,Gbm,HCN1,RP11-350G24.2_transcriptomics_ENSG00000258033.1,False,7.093768e-07
125,Gbm,HCN1,RP11-202K23.1_transcriptomics_ENSG00000233359.1,False,7.327203e-07
126,Gbm,HCN1,GOLGA8J_transcriptomics_ENSG00000179938.12,False,7.729804e-07


### Write Significant Comparisons (if any) to Shared CSV file

In [10]:
'''
existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)'''

"\nexisting_results = pd.read_csv(gene+'_Trans_Results.csv')\n\nupdated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)\n\nupdated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)"